In [1]:
from sympy import *
init_printing()

import numpy as np
import matplotlib.pyplot as plt

In [81]:

def intersect_cell(i,j,a,N,rho,phi):

    
    x0 = (2*a/N)*(i)-a
    x1 = (2*a/N)*(i)-a+(2*a/N)
    y0 = (2*a/N)*(j)-a
    y1 = (2*a/N)*(j)-a+(2*a/N)

    x,y= symbols("x,y")
    eq = Eq(x*cos(phi)+y*sin(phi),rho)
    eq

    #return (x0,y0),(x1,y1)

    left = False
    right = False
    top = False
    bottom = False
    diagonal = False

    
    #left
    if (len(solve(eq,y))!=0) and (y0 < solve(eq,y)[0].subs({x:x0}) < y1):
        left = True

    #right
    if (len(solve(eq,y))!=0) and (y0 < solve(eq,y)[0].subs({x:x1}) < y1):
        right = True
        
    #top
    if (len(solve(eq,x))!=0) and (x0 < solve(eq,x)[0].subs({y:y1}) < x1):
        top = True
        
    #bottom
    if (len(solve(eq,x))!=0) and (x0 < solve(eq,x)[0].subs({y:y0}) < x1):
        bottom = True

    #diagonals
    if rho == 0 and (phi == 3*pi/4 or phi == 7*pi/4):
        diagonal = True
    
    return left, right, top, bottom, diagonal
    
#intersect_cell(2,2,2,4,0.6,7*pi/4)
intersect_cell(0,3,2,4,0.3,5*pi/4)

(True, False, False, True, False)

In [80]:
phi = 5*pi/4
rho=0.3
a = 2
N = 4
i = 2
j = 1

x0 = (2*a/N)*(i)-a
x1 = (2*a/N)*(i)-a+(2*a/N)
y0 = (2*a/N)*(j)-a
y1 = (2*a/N)*(j)-a+(2*a/N)

x,y= symbols("x,y")
eq = Eq(x*cos(phi)+y*sin(phi),rho)
eq

print((x0,y0),(x1,y1))

solve(eq,y)[0].subs({x:x0})


(0.0, -1.0) (1.0, 0.0)


In [89]:

def get_length(i, j, a, N, rho, phi):

    x,y= symbols("x,y")
    eq = Eq(x*cos(phi)+y*sin(phi),rho)

    x0 = (2*a/N)*(i)-a
    x1 = (2*a/N)*(i)-a+(2*a/N)
    y0 = (2*a/N)*(j)-a
    y1 = (2*a/N)*(j)-a+(2*a/N)

    if (len(solve(eq,y))!=0):
        # left intersection
        vy = solve(eq,y)[0].subs({x:x0})
        vx = x0
        
        # right intersection
        hy = solve(eq,y)[0].subs({x:x1})
        hx = x1

    if (len(solve(eq,x))!=0):
        # top intersection
        tx = solve(eq,x)[0].subs({y:y1})
        ty = y1

        # bottom intersection
        bx = solve(eq,x)[0].subs({y:y0})
        by = y0

    index = intersect_cell(i, j, a, N, rho, phi)
    #print(index)

    if index[0] and index[1]:
        #print("left and right")
        vh = sqrt((vx-hx)**2+(vy-hy)**2)
        return vh

    if index[0] and index[2]:
        #print("left and top")
        vt = sqrt((vx-tx)**2+(vy-ty)**2)
        return vt
    
    if index[0] and index[3]:
        #print("left and bottom")
        vb = sqrt((vx-bx)**2+(vy-by)**2)
        return vb

    if index[1] and index[2]:
        #print("right and top")
        th = sqrt((tx-hx)**2+(ty-hy)**2)
        return th

    if index[1] and index[3]:
        #print("right and bottom")
        bh = sqrt((bx-hx)**2+(by-hy)**2)
        return bh

    if index[2] and index[3]:
        #print("top and bottom")
        tb = sqrt((tx-bx)**2+(ty-by)**2)
        return tb

    if index[4]:
        #print("diagonal")
        q0q1 = sqrt((x1-x0)**2+(y1-y0)**2)
        return q0q1
    

#print(get_length(0,2,2,4,0.3,5*pi/4))

In [105]:
def construct_A(a, N, rho_list, phi_list):
    rows_of_A = []  # This will store each flattened matrix as a row

    for laser in range(len(rho_list)):
        rho = rho_list[laser]
        phi = phi_list[laser]
        cell_values = []  # Initialize cell_values inside the loop to reset it for each laser

        for i in range(N):  # row
            for j in range(N):  # col
                length = get_length(i, j, a, N, rho, phi)
                cell_values.append(length if length is not None else 0)

        M = Matrix(N, N, cell_values)  # Create a matrix from the current list of cell values
        flattened_M = M.reshape(1, N*N)  # Flatten the matrix M to a single row
        rows_of_A.append(flattened_M)  # Add the flattened matrix as a new row in the list

    # Now, create the final matrix A using the list of rows
    A = Matrix(rows_of_A)  # Each element in rows_of_A is already a Matrix object

    return A

In [106]:
#print(intersect_cell(2,2,2,4,0,pi/4))

#print(get_length(2,2,2,4,0.1,5*pi/3))

construct_A(2, 4, [0.2,0.6,0.3,0.3], [4*pi/6,7*pi/4,8*pi/4,5*pi/4])


⎡        0          1.15470053837925   0           0           0   0.692820323
⎢                                                                             
⎢0.214213562373095         0           0           0          1.2  0.214213562
⎢                                                                             
⎢        0                 0           0           0           0           0  
⎢                                                                             
⎣        0                 0          0.6  0.814213562373095   0          0.6 

027551  0.461880215351701  0   0           0          1.15470053837925    0   
                                                                              
373095          0          0   0          1.2         0.214213562373095   0   
                                                                              
                0          0  1.0         1.0                1.0         1.0  
                                                   